In [3]:
from glob import glob
from ultralytics import YOLO
import os

In [2]:
model_for_sbd_mdt = YOLO('best_sbd_mdt.pt')
model_part2 = YOLO('best_part2.pt')

NameError: name 'YOLO' is not defined

In [169]:
test_image_files = glob(os.path.join('testset1/testset1', 'images', '*.jpg'))

result_path = os.path.join('submission', 'result1.txt')
print(test_image_files)

['testset1/testset1\\images\\IMG_3960_iter_0.jpg', 'testset1/testset1\\images\\IMG_3960_iter_1.jpg', 'testset1/testset1\\images\\IMG_3960_iter_10.jpg', 'testset1/testset1\\images\\IMG_3960_iter_100.jpg', 'testset1/testset1\\images\\IMG_3960_iter_101.jpg', 'testset1/testset1\\images\\IMG_3960_iter_102.jpg', 'testset1/testset1\\images\\IMG_3960_iter_103.jpg', 'testset1/testset1\\images\\IMG_3960_iter_104.jpg', 'testset1/testset1\\images\\IMG_3960_iter_105.jpg', 'testset1/testset1\\images\\IMG_3960_iter_106.jpg', 'testset1/testset1\\images\\IMG_3960_iter_107.jpg', 'testset1/testset1\\images\\IMG_3960_iter_108.jpg', 'testset1/testset1\\images\\IMG_3960_iter_109.jpg', 'testset1/testset1\\images\\IMG_3960_iter_11.jpg', 'testset1/testset1\\images\\IMG_3960_iter_110.jpg', 'testset1/testset1\\images\\IMG_3960_iter_111.jpg', 'testset1/testset1\\images\\IMG_3960_iter_112.jpg', 'testset1/testset1\\images\\IMG_3960_iter_113.jpg', 'testset1/testset1\\images\\IMG_3960_iter_114.jpg', 'testset1/testset

In [170]:
import cv2

In [171]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    
    x1, y1, w1, h1 = boxA
    x2, y2, w2, h2 = boxB

    boxA = [x1, y1, x1 - w1 / 2, y1 - h1 / 2]
    boxB = [x2, y2, x2 - w2 / 2, y2 - h2 / 2]

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # return the intersection over union value
    return iou

In [172]:
def remove_overlap(boxes, iou_threshold):
    boxes = sorted(boxes, 
                   key=lambda x: {x['box'][0], x['box'][1]}, 
                   )
    result = []
    for box in boxes:
        if len(result) == 0:
            result.append(box)
            continue

        pre = result[-1]
        if bb_intersection_over_union(pre['box'], box['box']) < iou_threshold:
            result.append(box)
    return result

In [173]:
def get_box(img, box):
    # yolo box format
    x, y, w, h = box

    x1 = int(x - w / 2)
    y1 = int(y - h / 2)

    x2 = int(x + w / 2)
    y2 = int(y + h / 2)

    img_cut = img[y1:y2, x1:x2]
    gray = cv2.cvtColor(img_cut, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cv2.imwrite('test.jpg', thresh)
    # return the ratio of white pixels
    print('result', thresh.sum() / (255 * w * h))
    return thresh.sum() / (255 * w * h)


In [174]:
def get_sbd(boxes, img):
    boxes = sorted(boxes, 
                   key=lambda x: {x['box'][0], x['box'][1]}, 
                   )
    
    result = []
    
    for i in range(0, 6):
        ticks = []
        col = boxes[i * 10 : (i + 1) * 10]
        col = sorted(col, key=lambda x: x['box'][1])
        print(len(col))
        for j in range(0, 10):
            if col[j]['label'] == 0:
                ticks.append(j)
                # print(boxes[j])
        print(ticks)

        if (len(ticks) == 1):
            print(f'SBD{i + 1}')
            result.append({f'SBD{i + 1}' : col[0]})
            continue

        if len(ticks) == 0:
            minConf = 10
            resId = 0
            for j in range(0, 10):
                if col[j]['conf'] < minConf:
                    minConf = col[j]['conf']
                    resId = j
        else:
            maxConf = 0
            resId = 0
            for j in ticks:
                if col[j]['conf'] > maxConf:
                    maxConf = col[j]['conf']
                    resId = j

        print(f'SBD{i + 1}')
        result.append({f'SBD{i + 1}' : col[resId]})

    return result
        



In [175]:
def get_mdt(boxes, img):
    boxes = sorted(boxes, 
                   key=lambda x: {x['box'][0], x['box'][1]}, 
                   )
    
    result = []
    
    for i in range(0, 3):
        ticks = []
        col = boxes[i * 10 : (i + 1) * 10]
        col = sorted(col, key=lambda x: x['box'][1])
        print(len(col))
        for j in range(0, 10):
            if col[j]['label'] == 0:
                ticks.append(j)
                # print(boxes[j])
        print(ticks)

        if (len(ticks) == 1):
            print(f'MDT{i + 1}')
            result.append({f'MDT{i + 1}' : col[0]})
            continue

        if len(ticks) == 0:
            minConf = 10
            resId = 0
            for j in range(0, 10):
                if col[j]['conf'] < minConf:
                    minConf = col[j]['conf']
                    resId = j
        else:
            maxConf = 0
            resId = 0
            for j in ticks:
                if col[j]['conf'] > maxConf:
                    maxConf = col[j]['conf']
                    resId = j

        print(f'MDT{i + 1}')
        result.append({f'MDT{i + 1}' : col[resId]})

    return result
        



In [176]:
def to_grid(boxes, numcol):
    left = 10 ** 9
    right = 0
    sumW = 0

    for box in boxes:
        x, y, w, h = box['box']

        x -= w / 2
        left = min(left, x)

        x += w
        right = max(right, x)

        sumW += w

    avgW = (right - left) / numcol

    result = []

    for i in range(0, numcol):
        # result.append([])
        col = []
        l = left + i * avgW
        r = left + (i + 1) * avgW
        for box in boxes:
            x, y, w, h = box['box']
            if x >= l and x <= r:
                col.append(box)
        
        result.append(col)

    return result

In [204]:
def to_grid_row(boxes, numrow):
    top = 10 ** 9
    bot = 0

    for box in boxes:
        x, y, w, h = box['box']

        y -= h / 2
        top = min(top, y)

        y += h
        bot = max(bot, y)


    avgW = (bot - top) / numrow

    result = []

    for i in range(0, numrow):
        # result.append([])
        col = []
        l = top + i * avgW
        r = top + (i + 1) * avgW
        for box in boxes:
            x, y, w, h = box['box']
            if y >= l and y <= r:
                col.append(box)
        
        result.append(col)

    return result

In [177]:
def get_ticked(col, top, bot, numRow):
    resid = 0

    ticked = []
    unticked = []

    avgH = (bot - top) / numRow
    if (len(col) == 0):
        # print('empty')
        return 0, None
    for i in range(0, numRow):
        t = top + i * avgH
        b = top + (i + 1) * avgH

        ls = []

        for j in range(0, len(col)):
            if col[j]['box'][1] >= t and col[j]['box'][1] <= b:
                if col[j]['label'] == 0:
                    ticked.append([i, col[j]['box'], col[j]['conf']])
                else:
                    unticked.append([i, col[j]['box'], col[j]['conf']])

    if len(ticked) > 0:
        mxConf = 0
        resid = 0
        
        bx = None

        for i in range(0, len(ticked)):
            if ticked[i][2] > mxConf:
                mxConf = ticked[i][2]
                resid = ticked[i][0]
                bx = ticked[i][1]

        return resid, bx
    
    mnConf = 10
    resid = 0

    bx = None

    for i in range(0, len(unticked)):
        if unticked[i][2] < mnConf:
            mnConf = unticked[i][2]
            resid = unticked[i][0]
            bx = unticked[i][1]

    return resid, bx

In [206]:
def get_ticked_row(col, left, right, numCol):
    resid = 0

    ticked = []
    unticked = []

    avgH = (right - left) / numCol
    if (len(col) == 0):
        # print('empty')
        return 0, None
    for i in range(0, numCol):
        t = left + i * avgH
        b = left + (i + 1) * avgH

        ls = []

        for j in range(0, len(col)):
            if col[j]['box'][0] >= t and col[j]['box'][0] <= b:
                if col[j]['label'] == 0:
                    ticked.append([i, col[j]['box'], col[j]['conf']])
                else:
                    unticked.append([i, col[j]['box'], col[j]['conf']])

    if len(ticked) > 0:
        mxConf = 0
        resid = 0
        
        bx = None

        for i in range(0, len(ticked)):
            if ticked[i][2] > mxConf:
                mxConf = ticked[i][2]
                resid = ticked[i][0]
                bx = ticked[i][1]

        return resid, bx
    
    mnConf = 10
    resid = 0

    bx = None

    for i in range(0, len(unticked)):
        if unticked[i][2] < mnConf:
            mnConf = unticked[i][2]
            resid = unticked[i][0]
            bx = unticked[i][1]

    return resid, bx

In [178]:
def draw_debug_(img, col, name):
    for box in col:
        x, y, w, h = box['box']
        x1 = int(x - w / 2)
        y1 = int(y - h / 2)

        x2 = int(x + w / 2)
        y2 = int(y + h / 2)

        if (box['label'] == 0):
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        else:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    cv2.imwrite(name, img)

In [179]:
def process_and_write_to_file(results, image_path):
    ticked_boxes = {
        'SBD1': '',
        'SBD2': '',
        'SBD3': '',
        'SBD4': '',
        'SBD5': '',
        'SBD6': '', 
        'MDT1': '',
        'MDT2': '',
        'MDT3': ''
    }

    # img = cv2.imread(image_path)
    WIDTH = 2255
    HEIGHT = 3151

    # print('dimension: ', WIDTH, HEIGHT)
    # for result in results:
    #     print(result.boxes)

    boxes = []

    # print(len(results.boxes))
    for box in results.boxes:
        boxes.append({
            'label': int(box.cls[0].item()),
            'box': box.xywh[0].tolist(),
            'conf': box.conf.item()
        })


    # print('len',len(boxes))
    
    top = 100000
    bot = 0

    sumH = 0

    for box in boxes:
        x, y, w, h = box['box']
        top = min(top, y - h / 2)
        bot = max(bot, y + h / 2)
        sumH += h

    avgH = sumH / len(boxes)
    
    grid = to_grid(boxes, 11)
    
    res_sbd_mdt = []
    for i in range(0, 6):
        col = grid[i]

        resid, bx = get_ticked(col, top, bot, 10)
        if bx == None:
            continue
        # res_sbd_mdt.append({f'SBD{i + 1}': {'box': bx, 'conf': boxes[resid]['conf']}})
        key = f'SBD{i + 1}'
        # print('box', bx)

        # print('get', key, resid)

        x = bx[0] / WIDTH
        y = bx[1] / HEIGHT
        w = bx[2] / WIDTH
        h = bx[3] / HEIGHT

        ticked_boxes[key] = f'{x:6f},{y:6f},{w:6f},{h:6f}'

    for i in range(8, 11):
        col = grid[i]

        resid, bx = get_ticked(col, top, bot, 10)
        if bx == None:
            continue
        # res_sbd_mdt.append({f'MDT{i - 7}': {'box': bx, 'conf': boxes[resid]['conf']}})
        key = f'MDT{i - 7}'
        
        # print('box', bx)
        # print('get', key, resid)

        x = bx[0] / WIDTH
        y = bx[1] / HEIGHT
        w = bx[2] / WIDTH
        h = bx[3] / HEIGHT

        ticked_boxes[key] = f'{x:6f},{y:6f},{w:6f},{h:6f}'


    return ticked_boxes


In [ ]:
# tmp_w = 2255
# tmp_h = 3151
# okpass = 0
# for image_path in test_image_files:
#     img = cv2.imread(image_path)
#     w, h = img.shape[1], img.shape[0]
#     if w != tmp_w or h != tmp_h:
#         print('wrong dimension')
#         continue
#     okpass += 1

# print('okpass', okpass)

In [198]:
part2_checklist = ['T', 'x', 'F', 'x', 'T', 'x', 'F', 'x', 'x', 'T', 'x', 'F', 'x', 'T', 'x', 'F', 'x', 'x', 'T', 'x', 'F', 'x', 'T', 'x', 'F', 'x', 'x', 'T', 'x', 'F', 'x', 'T', 'x', 'F' ]
print(len(part2_checklist))

34


In [229]:
def part2_process(results, image_path):
    ticked_boxes = {}
    for i in range(1, 9):
        ticked_boxes[f'2.{i}.a'] = ''
        ticked_boxes[f'2.{i}.b'] = ''
        ticked_boxes[f'2.{i}.c'] = ''
        ticked_boxes[f'2.{i}.d'] = ''


    WIDTH = 2255
    HEIGHT = 3151

    boxes = []

    # print(len(results.boxes))
    for box in results.boxes:
        boxes.append({
            'label': int(box.cls[0].item()),
            'box': box.xywh[0].tolist(),
            'conf': box.conf.item()
        })

    
    
    top = 100000
    bot = 0

    sumH = 0

    for box in boxes:
        x, y, w, h = box['box']
        top = min(top, y - h / 2)
        bot = max(bot, y + h / 2)
        sumH += h

    avgH = sumH / len(boxes)
    
    # grid = to_grid(boxes, 34)
    grid_row = to_grid_row(boxes, 4)

    option = ['a', 'b', 'c', 'd']

    opt = 0
    for row in grid_row:
        column = to_grid(row, 34)

        curLetter = option[opt]
        opt += 1
        quest = 0

        curBox = []

        for i in range(0, 34):
            if part2_checklist[i] == 'T':
                curBox = []
                quest += 1
                for tmpBox in column[i]:
                    curBox.append(tmpBox)
                continue

            if part2_checklist[i] == 'F':
                for tmpBox in column[i]:
                    curBox.append(tmpBox)
                left = 100000000
                right = 0

                for box in curBox:
                    x, y, w, h = box['box']
                    x -= w / 2
                    left = min(left, x)
                    x += w
                    right = max(right, x)

                resid, bx = get_ticked_row(curBox, left, right, 2)
                # print(len(curBox))
                # resid, bx = get_ticked(col, top, right, 10)
                # if bx == None:
                #     continue
                # # res_sbd_mdt.append({f'MDT{i - 7}': {'box': bx, 'conf': boxes[resid]['conf']}})
                
                key = f'2.{quest}.{curLetter}'
                
                # # print('box', bx)
                # # print('get', key, resid)

                x = bx[0] / WIDTH
                y = bx[1] / HEIGHT
                w = bx[2] / WIDTH
                h = bx[3] / HEIGHT

                ticked_boxes[key] = f'{x:6f},{y:6f},{w:6f},{h:6f}'

    return ticked_boxes


In [ ]:

num = 0

with open('results.txt', 'w') as f:

    for image_path in test_image_files:
        results_part2 = model_part2(image_path)
        res_part2 = part2_process(results_part2[0], image_path)
        # print(res_part2)
        # results = model_for_sbd_mdt (image_path)
        # answer = process_and_write_to_file(results[0], image_path)

        line = image_path.split('\\')[-1] + ' ' + ' '.join([f'{k} {v}' for k, v in res_part2.items()]) + '\n'
        f.write(line)


NameError: name 'test_image_files' is not defined